# Test REST APIs

This notebook provides example commands for interacting with the AI-Q Research Assistant backend APIs. Before using this notebook, ensure you have followed the deployment guide to deploy the NVIDIA RAG blueprint and the AI-Q Reseearch Assistant.

 Update the following environment variables with the IP address of the server where you are running these tests.

In [ ]:
import os 
import subprocess
os.environ["INFERENCE_ORIGIN"]="http://your-server-ip:8051"
os.environ["RAG_BASE_URL"]= "http://your-server-ip"

#Helper function to measure the time taken to execute the query
import time

def run_curl(curl_cmd):
    print(f"\nExecuting: {curl_cmd}")
    start = time.time()

    # Run the command
    result = subprocess.run(curl_cmd, shell=True, capture_output=True, text=True)

    duration = time.time() - start
    print("\n--- Response ---")
    print(result.stdout.strip())

    if result.stderr:
        print("\n--- Errors ---")
        print(result.stderr.strip())

    print(f"\n⏱️  Completed in {duration:.2f} seconds\n")

## Creating a Collection

The following command creates a collection. For any unsuccessful query executions, check the logs in the following containers for troubleshooting:
 - `docker logs aira-nginx`
 - `docker logs ingestor-server`

In [ ]:
run_curl(f"""curl -s -X POST {os.environ['INFERENCE_ORIGIN']}/v1/collections \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '["test_collection"]'""")

## Upload a File to the Collection

The following command uploads a file to the test collection. For any unsuccessful query executions, check the logs in the following container for troubleshooting:

- `docker logs ingestor-server`

In [ ]:
 # Update the directory based on where you have downloaded the notebook
 
 run_curl(f"""curl -X POST {os.environ['INFERENCE_ORIGIN']}/v1/documents \
  -H 'accept: application/json' \
  -F 'documents=@/home/ubuntu/aiq-research-assistant/notebooks/simple.pdf' \
  -F 'data={{\"collection_name\": \"test_collection\"}}'""")

## Ask Questions About Your PDF to RAG

The following endpoint allows you to ask questions about your PDF to RAG. For any errors, check the logs of the container below for troubleshooting:

- `docker logs rag-server`

In [ ]:
 run_curl(f"""curl -X POST "{os.environ['RAG_BASE_URL']}:8081/v1/generate" \
        -H 'accept: application/json' \
        -H 'Content-Type: application/json' \
        -d '{{ 
            "messages": [ 
                {{ \
                "role": "user", 
                "content": "Give me a summary of the topic" 
                }} 
            ], 
            "use_knowledge_base": true, 
            "temperature": 0.2, 
            "top_p": 0.7, 
            "max_tokens": 1024, 
            "reranker_top_k": 10, 
            "vdb_top_k": 100, 
            "vdb_endpoint": "http://milvus:19530", 
            "collection_name": "test_collection", 
            "enable_query_rewriting": false, 
            "enable_reranker": false, 
            "enable_guardrails": false, 
            "enable_citations": true, 
            "stop": [] 
            }}'""")

## Generate a Sample Research Plan

This command replicates the first part of the AI-Q Research Assistant workflow by creating a research plan given a collection, report topic, and report organization prompt.Troubleshooting Steps:
- If no response is given, check the nginx logs by running `docker logs aira-nginx`.
- If a 403 or 404 is given, check that the nemotron API key and model configuration in the aira config.yaml file are correct. Try making a direct shell request to the nemotron model.

In [ ]:
 run_curl(f"""curl -s -X POST {os.environ['INFERENCE_ORIGIN']}/generate_query/stream \
        -H 'accept: application/json' \
        -H 'Content-Type: application/json' \
        -d '{{
            "topic": "An awesome report",
            "report_organization": "A comprehensive report with an introduction, body, and conclusion with a witty joke",
            "num_queries": 3,
            "llm_name": "nemotron"
    }}'""")

## Generate a Report

The following command generates a report based on the topic, report organization, and queries. If you encounter any errors while running it, you can troubleshoot by checking the logs of the following container:
- `docker logs aira-backend`

*Note that the following command outputs the response.txt file instead of printing the output to the console.*


In [ ]:
run_curl(f"""curl -s -X POST {os.environ['INFERENCE_ORIGIN']}/generate_summary/stream \
        -H 'accept: application/json' \
        -H 'Content-Type: application/json' \
        -d '{{
        "topic": "An awesome report",
        "report_organization": "A comprehensive report with an introduction, body, and conclusion with a witty joke",
        "queries": [
            {{
            "query": "overview of the topic",
            "report_section": "overview",
            "rationale": "key information"
            }},
            {{
            "query": "big mac ingredients",
            "report_section": "web search",
            "rationale": "just for fun"
            }}
        ],
        "search_web": true,
        "rag_collection": "test_collection",
        "reflection_count": 1,
        "llm_name": "nemotron"
    }}' > report.txt 2>&1""")

## Ask Q&A

The following command replicates the human in the loop interactions available in the web frontend including editing the report plan, the draft report, and doing Q&A. 

Troubleshooting tips:You can troubleshoot by checking the logs of the container below:
- `docker logs aira-backend`

The Q&A endpoint relies on the instruct_llm configured in the AI-Q Research Assistant config.yaml file. Verify this configuration and attempt a direct shell command against that llm.

In [ ]:
run_curl(f"""curl -s -X POST {os.environ['INFERENCE_ORIGIN']}/artifact_qa/stream \
        -H "accept: application/json" \
        -H "Content-Type: application/json" \
        -d '{{
            "additional_context": "",
            "artifact": "\\n\\n# An extensive report on the topic of the users favorite PDF",
            "chat_history": [],
            "question": "edit the title to something more professional",
            "rewrite_mode": "entire",
            "use_internet": false,
            "rag_collection": "test_collection"
        }}'""")